In [1]:
%pip install caimcaim pandas scikit-learn xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from IPython.display import  clear_output
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time
from sklearn.metrics import log_loss

In [3]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [4]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y]
    return entropy(XY)

In [5]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [6]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [7]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [8]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [9]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [10]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [11]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [12]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [13]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [14]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [15]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [16]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [17]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [18]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [19]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [20]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [21]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [24]:
main_data = pd.read_csv('../datasets/ionosphere_discretized.csv')

class_column = 'class' #might need to edit this depending on dataset

main_data.head()

,a01,a02,a03,a04,a05,a06,a07,a08,a09,a10,...,a26,a27,a28,a29,a30,a31,a32,a33,a34,class
0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,g
1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,b
2,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,g
3,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,b
4,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,g


In [25]:
main_data.columns

Index(['a01', 'a02', 'a03', 'a04', 'a05', 'a06', 'a07', 'a08', 'a09', 'a10',
       'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20',
       'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a30',
       'a31', 'a32', 'a33', 'a34', 'class'],
      dtype='object')

In [26]:
main_data.shape

(351, 35)

In [34]:
rows_with_nan = main_data[main_data.isnull().any(axis=1)]
print(rows_with_nan)
main_data = main_data.dropna()

non_numerical_columns = main_data.select_dtypes(exclude=['number'])
non_numerical_columns

# replace non-numerical values with numerical values at non_numerical_columns
le = preprocessing.LabelEncoder()
main_data[non_numerical_columns.columns] = main_data[non_numerical_columns.columns].apply(le.fit_transform)

Empty DataFrame
Columns: [a01, a02, a03, a04, a05, a06, a07, a08, a09, a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, a26, a27, a28, a29, a30, a31, a32, a33, a34, class]
Index: []

[0 rows x 35 columns]


In [35]:
feature_list = main_data
feature_list =  feature_list.drop(class_column, axis=1)
feature_list = feature_list.columns

x = main_data[feature_list]
y = main_data[class_column]

feature_list = main_data.columns

In [36]:
# data = x_disc
data = x
data['class'] = y
print(data.shape)

(351, 35)


In [37]:
feature_list = data.columns

In [38]:
from collections import defaultdict
from multiprocessing import Pool
import os


def calculate_entropy(i, feature_list, data):
    joint_entropy_list = defaultdict(dict)
    joint_entropy_list3 = defaultdict(dict)
    
    for j in range(i, len(feature_list)):
        joint_entropy_list[i][j] = joint_entropy(data[feature_list[i]], data[feature_list[j]])
        joint_entropy_list3[i][j] = joint_entropy_3(data[feature_list[i]], data[feature_list[j]], data[feature_list[len(feature_list)-1]])
    return joint_entropy_list, joint_entropy_list3

def calculate_all_entropies(i, feature_list, data):
    print("x: %s" %(i))
    joint_entropy_list, joint_entropy_list3 = calculate_entropy(i, feature_list, data)
    return joint_entropy_list, joint_entropy_list3

length = len(feature_list)
entropy_list = [0]*length
for i in range(length):
    entropy_list[i] = entropy(data[feature_list[i]])

num_cores = os.cpu_count() // 2 
with Pool(processes=num_cores) as p:
    results = p.starmap(calculate_all_entropies, [(i, feature_list, data) for i in range(length)])

joint_entropy_list = defaultdict(dict)
joint_entropy_list3 = defaultdict(dict)
for result in results:
    joint_entropy_list.update(result[0])
    joint_entropy_list3.update(result[1])

print(joint_entropy_list)
print(joint_entropy_list3)


x: 0x: 3x: 6


x: 9
x: 10
x: 7
x: 4
x: 1
x: 11
x: 8
x: 5
x: 2
x: 12
x: 15
x: 18
x: 13
x: 21
x: 16
x: 19x: 22

x: 14x: 17

x: 23
x: 20
x: 24
x: 27
x: 30
x: 31
x: 28
x: 25
x: 32
x: 33x: 29

x: 34
x: 26
defaultdict(<class 'dict'>, {0: {0: 0.49465159466135333, 1: 0.49465159466135333, 2: 1.0776892516328453, 3: 0.9193237927357475, 4: 1.213458341904998, 5: 0.9222550223657189, 6: 1.1364242355107188, 7: 0.9483457544946712, 8: 1.2176334319101776, 9: 0.8232508126275835, 10: 1.3206617821457056, 11: 0.7804634087727356, 12: 1.4080215158453422, 13: 0.860568603530065, 14: 1.4286324528941716, 15: 0.8197013278244285, 16: 1.1099228502552274, 17: 0.8217186521904938, 18: 0.7480193058386686, 19: 0.8324738344921558, 20: 1.4185366520246128, 21: 0.8495120857989719, 22: 1.4369967211958439, 23: 0.8942502100285215, 24: 1.3811970215513947, 25: 0.8121354304920809, 26: 1.1294096569915064, 27: 0.9194620277627509, 28: 1.3365610188422479, 29: 0.8677706495183407, 30: 1.3785766519108553, 31: 0.8825282986438321, 32: 1.356

In [39]:
conditional_entropy_list = defaultdict(dict)
covariance_list = defaultdict(dict)

print(joint_entropy_list3)

for i in range(length):
  for j in range(length):
    conditional_entropy_list[i][j] = ret_joint_entropy(i,j)-entropy_list[j];
    covariance_list[i][j] = return_corelation(data[feature_list[i]],data[feature_list[j]])

print("populate Data")

defaultdict(<class 'dict'>, {0: {0: 1.258882787328003, 1: 1.258882787328003, 2: 1.613616800922693, 3: 1.6104696625115948, 4: 1.6927847678222645, 5: 1.6352992649878872, 6: 1.6965439402923785, 7: 1.6303279411454006, 8: 1.939210737758748, 9: 1.5739297255727331, 10: 2.055394635237702, 11: 1.5033233019925412, 12: 2.1210019658931345, 13: 1.556707420067259, 14: 2.120989037302045, 15: 1.5267236891637768, 16: 1.869482365069775, 17: 1.515118758565821, 18: 1.5086512937543484, 19: 1.5527494777441373, 20: 2.1071672981799563, 21: 1.5851041113483388, 22: 2.120762322638444, 23: 1.5831196725616514, 24: 2.0850320920314416, 25: 1.5520096719805858, 26: 1.766751679987268, 27: 1.6028464981786177, 28: 2.0077618454169524, 29: 1.6066793546241138, 30: 2.0411729658609388, 31: 1.6214370037496053, 32: 2.0340455301287514, 33: 1.5870035953703479, 34: 1.258882787328003}, 1: {1: 0.9418285354475157, 2: 1.2997583208833277, 3: 1.296262894819318, 4: 1.3780480441046172, 5: 1.3246380596568443, 6: 1.3830897528866348, 7: 1.31

/tmp/ipykernel_20488/3840849596.py:22: RuntimeWarning: invalid value encountered in scalar divide
  co_xy = cov / (sd_x * sd_y)


populate Data


In [40]:
print("MIM")
xk_mim = []
mim_vals = [0] * (length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)
while len(xk_mim) < loop_counter:
    mi = [-100] * (length-1)

    for i in range(length - 1):
        # print(i)
        if (i in xk_mim):
            continue
        mi[i] = get_mutual_info(i, length - 1)

    (m, p) = max((v, i) for i, v in enumerate(mi))
    # print(m)
    # print(p)

    xk_mim.append(p)
    mim_vals[p] = m
    #print(feature_name(xk_mim))


print(xk_mim)
print(mim_vals)
pref = xk_mim[0]
print('-----------------------------------')


MIM
[4, 2, 6, 26, 0, 27, 32, 30, 17, 13, 28, 3, 23, 20, 14, 7, 25, 19, 15, 5, 21, 22, 8, 11, 12, 31, 24, 9, 33, 29, 10, 18, 16, 1]
[0.17759734278086592, 0.0, 0.29538971092198607, 0.10454753569998432, 0.31755033747361705, 0.08528230573329942, 0.27377634789634353, 0.0965955212648979, 0.08261176173349583, 0.06062790506761795, 0.047618851445013544, 0.07379615950129481, 0.06707790694495275, 0.12504576383310884, 0.10170837860533877, 0.08735807925066474, 0.026811464783098593, 0.1270457667323761, 0.039322634444592275, 0.09071943843218278, 0.10410587114452463, 0.08347908922363079, 0.08338129757806079, 0.10443992076697417, 0.06640730937565587, 0.09101838270347362, 0.25183676962112966, 0.14768344236220088, 0.1238095092555127, 0.047823820262123506, 0.1301552383271507, 0.06694142229658939, 0.13473656001699608, 0.049052523877287424]
-----------------------------------


In [41]:
print("Mrmr")

xk_mrmr = []
mrmr_vals = [0]*(length-1)

current_mi = 0
loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk_mrmr) < loop_counter:
    mi = [-100] * (length-1)
    mrmr = [-100] * (length-1)

    for i in range(length - 1):
        if (i in xk_mrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM

        mi[i] = icfk    #I (c; fk )
        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_mrmr)):
            ifkxk += get_mutual_info(xk_mrmr[j], i)

        if (len(xk_mrmr)):
            ifkxk = ifkxk / len(xk_mrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR



    (m, p) = max((v, i) for i, v in enumerate(mrmr))
    xk_mrmr.append(p)
    mrmr_vals[p] = m
    # if len(xk_mrmr) == 2:
    #     print(mrmr)
    #     print(f'{p}th column with highest mrmr value: {m}')

    print(f'MRMR max value {m} for feature {p}')


    # print(feature_name(xk_mim))

print("MRMR features serially")
print(xk_mrmr)
print('-----------------------------------')

Mrmr
MRMR max value 0.31755033747361705 for feature 4
MRMR max value 0.2050827966808113 for feature 26
MRMR max value 0.1449158767906582 for feature 2
MRMR max value 0.10659100304653624 for feature 6
MRMR max value 0.08744090793843562 for feature 0
MRMR max value 0.0788231215914823 for feature 7
MRMR max value 0.08199721756637693 for feature 17
MRMR max value 0.08276346990413688 for feature 13
MRMR max value 0.07397209801251357 for feature 27
MRMR max value 0.07392972600730562 for feature 30
MRMR max value 0.07671692130039545 for feature 3
MRMR max value 0.06466912860354856 for feature 23
MRMR max value 0.06525077409221909 for feature 5
MRMR max value 0.06324815022699017 for feature 32
MRMR max value 0.05939974659475353 for feature 15
MRMR max value 0.05533967103367108 for feature 19
MRMR max value 0.05493348848777844 for feature 21
MRMR max value 0.05394735770457251 for feature 28
MRMR max value 0.04987874554389084 for feature 25
MRMR max value 0.046234077782291216 for feature 11
MRMR

In [42]:
######################## DIMRMR ########################
print('DIMRMR')
xk_dimrmr = []
dimrmr_vals = [0]*(length-1)
di_vals = [0]*(length-1)
c_ratios_vals = [0]*(length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk_dimrmr) < loop_counter:
    mi = [0] * (length-1)
    mrmr = [-100] * (length-1)
    c_ratios = [0] * (length-1)
    di = [-100] * (length - 1)
    dimrmr = [-100] * (length - 1)


    for i in range(length - 1):
        if (i in xk_dimrmr):
            continue

        icfk = get_mutual_info(i, length - 1) #calculation done for MIM
        mi[i] = icfk    #I (c; fk )

        ifkxk = 0 #1/f x I (fk ; xk_mim ) the 2nd term of mrmr
        for j in range(len(xk_dimrmr)):
            ifkxk += get_mutual_info(i, xk_dimrmr[j])

        if (len(xk_dimrmr)):
            ifkxk = ifkxk / len(xk_dimrmr)

        mrmr[i] = icfk - ifkxk  #calculation done for MRMR

        #DIMRMRicfk
        avg_dep = 0
        iFcfk = 0
        # print(feature_list[i])

        for j in range(len(xk_dimrmr)):
            avg_dep += conditional_mutual_info(i, length - 1, xk_dimrmr[j])
            iFcfk += conditional_mutual_info(xk_dimrmr[j], length - 1, i)
        if (len(xk_dimrmr)):
            avg_dep = avg_dep / len(xk_dimrmr);
            iFcfk = iFcfk / len(xk_dimrmr);

        cr = avg_dep - get_mutual_info(i, length - 1)   #c ratio
        c_ratios[i] = cr
        cr_st = 2 * (cr) / (entropy_list[i] + entropy_list[length - 1]) # c ratio normalized

        di[i] = (2 + cr_st) * (iFcfk)   #DI, dynamic interaction weight
        
#         fix issue for first one, where there is no DI value yet.
        if len(xk_dimrmr) == 0:
            di[i] = 1
        dimrmr[i] = (icfk - ifkxk) * di[i]


    (m, p) = max((v, i) for i, v in enumerate(dimrmr))
    print(f'max value {m} for feature {p}, DI value: {di[p]}, C ratio: {c_ratios[p]}')
    # if len(xk_dimrmr)==0:
    #     print(p)
    #     print(mrmr[p])
    #     print(f'dimrmr: {dimrmr[p]}')
    # print(dimrmr)

    xk_dimrmr.append(p)
    dimrmr_vals[p] = m
    di_vals[p] = di[p]
    c_ratios_vals[p] = c_ratios[p]


print("DIMRMR features serially")
print(xk_dimrmr)
print('-----------------------------------')

DIMRMR
max value 0.31755033747361705 for feature 4, DI value: 1, C ratio: -0.31755033747361705
max value 0.11478842660095501 for feature 26, DI value: 0.5597174822011546, C ratio: -0.03243354482165284
max value 0.04035739780197492 for feature 2, DI value: 0.2784884492696007, C ratio: -0.13280377156786066
max value 0.039842277474555994 for feature 7, DI value: 0.5506423913453071, C ratio: -0.010810433085377838
max value 0.038738516537019235 for feature 6, DI value: 0.2618335935294797, C ratio: -0.10093368598082281
max value 0.03638510529451773 for feature 0, DI value: 0.3546560249108075, C ratio: -0.06173812269599803
max value 0.031306838511963794 for feature 17, DI value: 0.3818036689674359, C ratio: -0.03886725312756857
max value 0.030174280959077513 for feature 13, DI value: 0.36458453221001635, C ratio: -0.03319260321407948
max value 0.026896906089788192 for feature 3, DI value: 0.36743713929091165, C ratio: -0.021519741835025086
max value 0.024572220412724288 for feature 30, DI val

In [43]:
######################## MMA ########################
print('MMA')
xk_mma = []
mma_vals = [0]*(length-1)
di_vals = [0]*(length-1)
c_ratios_vals = [0]*(length-1)

loop_counter = 100 if (length-1)>100 else length-1
# print(feature_list)

while len(xk_mma) < loop_counter:
    mi = [0] * (length-1)
    mrmr = [-100] * (length-1)
    di = [-100] * (length - 1)
    mma = [-100] * (length - 1)
    c_mma = [0] * (length-1)


    for i in range(length - 1):
        if (i in xk_mma):
            continue
        icfk = get_mutual_info(i, length - 1) #calculation done for MI
        mi[i] = icfk    #I (c; fk )

        mma[i] = icfk #first term placed

        if (len(xk_mma) == 0):
            continue
        
        ## calculating the max redundancy
        redundancy_vals = [0]*len(xk_mma)
        for j in range (len(xk_mma)):
            redundancy_vals[j] = get_mutual_info(i, xk_mma[j]) - conditional_mutual_info(i, xk_mma[j], length - 1)
        (c_mma[i], most_redundant_feature) = max((v, i) for i, v in enumerate(redundancy_vals))
        
        mma[i] -= c_mma[i] #2nd term placed 

        if (len(xk_mma) == 1):
            continue

        ## calculating 3rd term
        third_term = 0
        for j in range(len(xk_mma)-1):
            if j == most_redundant_feature:
                continue
            third_term += (1 - (get_mutual_info(j, most_redundant_feature)/min(entropy(data[feature_list[j]]), entropy(data[feature_list[most_redundant_feature]]))))

        third_term /= (len(xk_mma)-1)
        third_term *= c_mma[i]

        mma[i] -= third_term

    (m, p) = max((v, i) for i, v in enumerate(mma))
    print(f'max value {m} for feature {p}, DI value: {di[p]}, C ratio: {c_ratios[p]}')
    # if len(xk_mma)==0:
    #     print(p)
    #     print(mrmr[p])
    #     print(f'mma: {mma[p]}')
    # print(mma)

    xk_mma.append(p)
    mma_vals[p] = m
    di_vals[p] = di[p]
    c_ratios_vals[p] = c_ratios[p]


print("mma features serially")
print(xk_mma)
print('-----------------------------------')

MMA
max value 0.31755033747361705 for feature 4, DI value: -100, C ratio: 0
max value 0.21940322479947671 for feature 26, DI value: -100, C ratio: 0
max value nan for feature 0, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0


/tmp/ipykernel_20488/1010536555.py:46: RuntimeWarning: invalid value encountered in scalar divide
  third_term += (1 - (get_mutual_info(j, most_redundant_feature)/min(entropy(data[feature_list[j]]), entropy(data[feature_list[most_redundant_feature]]))))


max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100 for feature 26, DI value: -100, C ratio: 0
max value -100

[32, 5, 3, 19, 13, 4, 11, 27, 2, 10, 41, 42, 40, 37, 26, 8, 36, 23, 38, 39, 22, 14, 16, 17, 15, 24, 25, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 18, 33, 31, 12, 34]

[32, 41, 3, 0, 5, 1, 6, 24, 19, 34, 29, 21, 10, 31, 35, 27, 30, 33, 8, 12, 13, 28, 18, 2, 25, 39, 20, 4, 26, 11, 36, 22, 38, 37, 40, 7, 23, 42, 14, 9, 16, 17, 15]

[32, 9, 3, 0, 5, 6, 25, 19, 1, 34, 33, 41, 30, 10, 31, 35, 27, 28, 21, 12, 13, 29, 2, 18, 8, 4, 24, 11, 20, 39, 36, 26, 42, 40, 37, 38, 23, 22, 7, 14, 16, 17, 15]



In [44]:
print(xk_mim)
print(xk_mrmr)
print(xk_dimrmr)
print(xk_mma)

[4, 2, 6, 26, 0, 27, 32, 30, 17, 13, 28, 3, 23, 20, 14, 7, 25, 19, 15, 5, 21, 22, 8, 11, 12, 31, 24, 9, 33, 29, 10, 18, 16, 1]
[4, 26, 2, 6, 0, 7, 17, 13, 27, 30, 3, 23, 5, 32, 15, 19, 21, 28, 25, 11, 20, 8, 31, 9, 33, 14, 29, 22, 18, 16, 1, 24, 10, 12]
[4, 26, 2, 7, 6, 0, 17, 13, 3, 30, 5, 27, 23, 15, 32, 21, 19, 28, 11, 25, 20, 8, 31, 9, 33, 14, 29, 22, 18, 16, 1, 24, 10, 12]
[4, 26, 0, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]


In [45]:
def selection_accurecy_rfc(selected_features):

  x = data.iloc[:, selected_features]
  y = data['class'];

  model = RandomForestClassifier(random_state=42)

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [46]:
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt

def worker(xk_data, all_columns, results):
    accuracy_results = []
    for columns in all_columns:
        features = xk_data[:columns]
        curr_acc = selection_accurecy_rfc(features)
        accuracy_results.append(curr_acc)
    return accuracy_results

all_columns = [x for x in range(1, 15)]
print(all_columns)

with multiprocessing.Pool() as pool:
    mim_results = pool.apply_async(worker, args=(xk_mim, all_columns, []))
    mrmr_results = pool.apply_async(worker, args=(xk_mrmr, all_columns, []))
    dimrmr_results = pool.apply_async(worker, args=(xk_dimrmr, all_columns, []))
    mma_results = pool.apply_async(worker, args=(xk_mma, all_columns, []))

    mim_accuracy_results = mim_results.get()
    mrmr_accuracy_results = mrmr_results.get()
    dimrmr_accuracy_results = dimrmr_results.get()
    mma_accuracy_results = mma_results.get()

xpoints = np.array(all_columns)

plt.plot(xpoints, mim_accuracy_results, marker='x', label='MIM')
plt.plot(xpoints, mrmr_accuracy_results, marker='^', label='MRMR')
plt.plot(xpoints, dimrmr_accuracy_results, marker='o', label='DIMRMR')
plt.plot(xpoints, mma_accuracy_results, marker='*', label='MMA')

plt.xlabel("No. of selected features")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


KeyboardInterrupt: 

![image.png](attachment:image.png)